In [1]:
import torch
from torch import nn
from torch.nn import functional as F

# 首先回顾一下多层感知机
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
# nn.Sequential定义了一种特殊的Module
X = torch.rand(2, 20)
net(X)

tensor([[-0.3097, -0.0372,  0.3381,  0.0987, -0.0319,  0.0395, -0.1590,  0.2137,
         -0.0350, -0.0658],
        [-0.0889, -0.0079,  0.3512, -0.0652, -0.0796,  0.0355, -0.1300,  0.1408,
          0.0440, -0.1611]], grad_fn=<AddmmBackward0>)

In [2]:
"""
Module在pytorch中是一个非常重要的抽象类
"""


# 自定义块(Module)
# 最重要的是两个函数:__init__和forward
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 两个全连接层,一个h,一个out
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        # 输入X,return output
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.0807,  0.1049, -0.0275,  0.1467,  0.0886, -0.0606,  0.1285,  0.0724,
         -0.1141, -0.1272],
        [ 0.0955, -0.0367, -0.0425,  0.0582,  0.0342, -0.1766,  0.0266,  0.0064,
         -0.0095, -0.1258]], grad_fn=<AddmmBackward0>)

In [4]:
# 自己实现Sequential(顺序块)
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # nn.Linear,nn.ReLU等等block
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X


net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.2667, -0.1907, -0.5101, -0.0984,  0.1334, -0.0597, -0.0566,  0.3291,
          0.0235, -0.1745],
        [ 0.1626, -0.1257, -0.4535, -0.1122,  0.1661, -0.0944, -0.2050,  0.2493,
          0.1379,  0.0561]], grad_fn=<AddmmBackward0>)

In [5]:
# 在forward函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        # 自定义,随便写的
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() < 1:
            X /= 2
        return X.sum()


net = FixedHiddenMLP()
net(X)

tensor(-0.7163, grad_fn=<SumBackward0>)

In [6]:
# 混合搭配各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

# chimera 翻译:混合,嵌合体
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-3.2797, grad_fn=<SumBackward0>)